In [1]:
import pandas as pd

In [5]:
song_ids = pd.read_pickle("song_ids.pickle")

In [115]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
cid ="186e8d1297e24171a54f8cff036b0426" 
secret = "033eaa62aa4a4383bd2102e5aa83ea6f"
username = "mcbabe17"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private playlist-modify-private user-modify-playback-state playlist-read-collaborative playlist-modify-public user-top-read user-read-currently-playing app-remote-control playlist-modify-private user-library-read'
token = util.prompt_for_user_token(username, scope, cid, secret, 'http://localhost:8080')
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [116]:
track = sp.track(song_ids[4])
track['name'],0 if track['explicit'] == False else 1

('Come Get Her', 1)

In [75]:
[item['name'] for item in sp.track(song_ids[1001])['artists']]

['Hopsin', 'Jarren Benton']

In [114]:
song_ids[323]

'0Rw5Go9iYcvxun5DhSdroI'

In [113]:
len(song_ids[75:])

11725

In [11]:
sp.audio_analysis(song_ids[0])['track'].keys()

dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])

In [36]:
len(sp.audio_analysis(song_ids[0])['sections']) # number of sections in a song, 

# Sections are defined by large variations in rhythm or timbre, e.g. chorus, verse, bridge, guitar solo,
# etc. Each section contains its own descriptions of tempo, key, mode, time_signature, and loudness.

# I think this is the only feature i'll use from the audio analysis API call. I don't expect the smaller nuances
# like segments, or length of beats will matter as much for popularity of songs

8

In [20]:
len(sp.audio_analysis(song_ids[0])['segments'])

989

In [51]:
test = {}


In [79]:
sp.audio_features(song_ids[1000:1050])

[{'danceability': 0.692,
  'energy': 0.668,
  'key': 1,
  'loudness': -11.357,
  'mode': 1,
  'speechiness': 0.373,
  'acousticness': 0.135,
  'instrumentalness': 0,
  'liveness': 0.162,
  'valence': 0.199,
  'tempo': 129.74,
  'type': 'audio_features',
  'id': '6xCZ5hulrqJwPb4CPK4e7G',
  'uri': 'spotify:track:6xCZ5hulrqJwPb4CPK4e7G',
  'track_href': 'https://api.spotify.com/v1/tracks/6xCZ5hulrqJwPb4CPK4e7G',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6xCZ5hulrqJwPb4CPK4e7G',
  'duration_ms': 272680,
  'time_signature': 4},
 {'danceability': 0.666,
  'energy': 0.917,
  'key': 7,
  'loudness': -3.602,
  'mode': 1,
  'speechiness': 0.322,
  'acousticness': 0.104,
  'instrumentalness': 0,
  'liveness': 0.457,
  'valence': 0.732,
  'tempo': 139.915,
  'type': 'audio_features',
  'id': '0dVvy5f7LbyTEsXGCyaNfY',
  'uri': 'spotify:track:0dVvy5f7LbyTEsXGCyaNfY',
  'track_href': 'https://api.spotify.com/v1/tracks/0dVvy5f7LbyTEsXGCyaNfY',
  'analysis_url': 'https://api.spotify.

In [56]:
feature_bad_keys = ['type', 'id', 'uri', 'track_href', 'analysis_url']

### Example row of data will have the following:

In [91]:
## Note, I could have plugged in a list of song_ids and gotten 99% of my data points, but the "sections"
## data is only available via a single track search. So, because of this one field, I'm must going to get all my
## data for each row on a 1 call per song_id basis since I have to make a call for each song_id's sections anyway :(((  

rows = []

for i in range(1000,1050):
    row = {}
    row['id'] = sp.track(song_ids[i])['id']
    for key, value in sp.audio_features(song_ids[i])[0].items():
        if key not in feature_bad_keys:
            row[key] = value
    row['num_sections'] = len(sp.audio_analysis(song_ids[i])['sections'])
    row['explicit'] = 0 if sp.track(song_ids[i])['explicit'] == False else 1

    row['popular'] = 0 if sp.track(song_ids[i])['popularity'] < 50 else 1
    
    rows.append(row)



In [98]:
rows[0]

{'id': '6xCZ5hulrqJwPb4CPK4e7G',
 'danceability': 0.692,
 'energy': 0.668,
 'key': 1,
 'loudness': -11.357,
 'mode': 1,
 'speechiness': 0.373,
 'acousticness': 0.135,
 'instrumentalness': 0,
 'liveness': 0.162,
 'valence': 0.199,
 'tempo': 129.74,
 'duration_ms': 272680,
 'time_signature': 4,
 'num_sections': 17,
 'explicit': 1,
 'popular': 0}

In [96]:
pd.DataFrame(rows).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                50 non-null     object 
 1   danceability      50 non-null     float64
 2   energy            50 non-null     float64
 3   key               50 non-null     int64  
 4   loudness          50 non-null     float64
 5   mode              50 non-null     int64  
 6   speechiness       50 non-null     float64
 7   acousticness      50 non-null     float64
 8   instrumentalness  50 non-null     float64
 9   liveness          50 non-null     float64
 10  valence           50 non-null     float64
 11  tempo             50 non-null     float64
 12  duration_ms       50 non-null     int64  
 13  time_signature    50 non-null     int64  
 14  num_sections      50 non-null     int64  
 15  explicit          50 non-null     int64  
 16  popular           50 non-null     int64  
dtyp

In [97]:
rows[0]

{'id': '6xCZ5hulrqJwPb4CPK4e7G',
 'danceability': 0.692,
 'energy': 0.668,
 'key': 1,
 'loudness': -11.357,
 'mode': 1,
 'speechiness': 0.373,
 'acousticness': 0.135,
 'instrumentalness': 0,
 'liveness': 0.162,
 'valence': 0.199,
 'tempo': 129.74,
 'duration_ms': 272680,
 'time_signature': 4,
 'num_sections': 17,
 'explicit': 1,
 'popular': 0}